# 1D Hallway Robot Localization with Kalman Filter

In this simulation, we implement a 1D hallway robot localization using a Kalman filter. The Kalman filter is a recursive algorithm used to estimate the state of a linear dynamic system based on noisy measurements.

## Simulation Overview

The simulation consists of the following components:
- Continuous-time dynamics matrices (`A` and `B`)
- Measurement matrix (`H`)
- Kalman filter parameters (initial values for process noise covariance `Q_initial` and measurement noise covariance `R_initial`)
- Initial state (`x_initial`) and uncertainty (`P_initial`)
- Time step (`dt`)
- Measurement function (`measurement_func`)
- Kalman filter prediction and update functions (`kalman_filter_prediction_update`)
- Functions to calculate precision and accuracy (`calculate_precision` and `calculate_accuracy`)
- Simulation function (`simulate_robot`)
- Interactive widget for noise level adjustment

## Kalman Filter Working Explanation

The Kalman filter operates in two main steps: prediction and update.

### Prediction Step
- Predicts the next state of the system based on the current state and control inputs.
- Uses the state transition matrix `A`, input matrix `B`, and control input `u`.

### Update Step
- Incorporates new measurements to update the state estimate.
- Calculates the Kalman gain, updates the state estimate, and adjusts the covariance matrix.

### Kalman Gain (\( K \))

The Kalman gain is a crucial component of the Kalman filter update step. It determines the blending of the predicted state estimate with the new measurement, balancing the trustworthiness of the prediction and the reliability of the measurement.

The Kalman gain (`K`) is calculated using the following equation:

$K = P_{\text{pred}} \cdot H^T \cdot (H \cdot P_{\text{pred}} \cdot H^T + R)^{-1}$

Where:
- \( K \) is the Kalman gain.
- $(P_{\text{pred}})$ is the predicted state covariance matrix.
- \( H \) is the measurement matrix.
- \( R \) is the measurement noise covariance matrix.

A higher Kalman gain assigns more weight to the new measurement, updating the state estimate more towards the measurement. Conversely, a lower Kalman gain indicates greater confidence in the predicted state estimate, leading to less adjustment based on the measurement.

## Code Implementation

The code includes functions to simulate the robot localization process, visualize the true and filtered paths, and interactively adjust the noise levels.

## Results and Visualization

The simulation produces an animation showing the true path, noisy measurements, and filtered path of the robot over time. Additionally, precision (covariance) and accuracy (mean squared error) metrics are calculated to evaluate the performance of the Kalman filter.

## Conclusion

The Kalman filter effectively estimates the position of the robot in the 1D hallway environment, demonstrating its utility in state estimation tasks with noisy sensor measurements.

## Kalman Filter Simulation Results

In this simulation, we use a Kalman filter to estimate the position of a robot moving in a 1-dimensional hallway. The Kalman filter combines information from the robot's motion model and noisy sensor measurements to provide an accurate estimate of the robot's position over time.

## Plot Components

### True Path
The true path represents the actual position of the robot over time without any noise or errors. It serves as the ground truth trajectory of the robot.

### Measurements
The measurements are noisy observations obtained from sensors. These measurements contain errors and inaccuracies, which are represented by the red dots in the plot.

### Filtered Path
The filtered path is the estimated position of the robot after incorporating both the motion model predictions and the noisy measurements using the Kalman filter. It represents the best estimate of the robot's position based on available measurements and the dynamics of the system.

By comparing these three components, we can observe how the Kalman filter improves the accuracy of the estimated position compared to relying solely on the motion model or sensor measurements.

## Conclusion
The Kalman filter demonstrates its effectiveness in accurately estimating the position of the robot, even in the presence of noisy sensor measurements and uncertainties in the motion model. It provides a robust method for state estimation in dynamic systems.


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Continuous-time dynamics matrices
A = np.array([[1, 1, 0.5],   # State transition matrix
              [0, 1, 1],
              [0, 0, 1]])

B = np.array([[1/6], [0.5], [1]])  # Input matrix

# Measurement matrix
H = np.array([[1, 0, 0]])

# Kalman filter parameters (initial values)
Q_initial = np.array([[1e-2, 0, 0],   # Process noise covariance matrix
                      [0, 1e-1, 0],
                      [0, 0, 1]])

R_initial = 5  # Measurement noise covariance

# Initial state
x_initial = np.array([[0], [0], [0]])  # Initial state [position, velocity, acceleration]
P_initial = np.diag([1, 1, 1])  # Initial uncertainty

# Time step
dt = 0.01  # Reduced time step for finer granularity

# Measurement function
def measurement_func(x):
    return np.dot(H, x) + np.random.normal(0, R_initial)

# Continuous-time Kalman filter prediction and update
def kalman_filter_prediction_update(x, P, z, Q, R, dt):
    # Prediction
    x_pred = np.dot(A, x)
    P_pred = np.dot(np.dot(A, P), A.T) + Q

    # Update
    K = np.dot(np.dot(P_pred, H.T), np.linalg.inv(np.dot(np.dot(H, P_pred), H.T) + R))
    x = x_pred + np.dot(K, (z - np.dot(H, x_pred)))
    P = np.dot((np.eye(len(x)) - np.dot(K, H)), P_pred)

    return x, P

# Function to calculate precision (covariance)
def calculate_precision(P):
    return np.diag(P)

# Function to calculate accuracy (mean squared error)
def calculate_accuracy(true_path, filtered_path):
    return np.mean((true_path - filtered_path)**2)

# Simulation function
def simulate_robot(Q_value, R_value):
    # Set noise covariance matrices
    Q = Q_value * Q_initial
    R = R_value * R_initial

    # Initialize state and uncertainty
    x = x_initial.copy()
    P = P_initial.copy()

    # Simulation parameters
    num_steps = 1000  # Increased number of simulation steps for finer granularity
    true_path = np.zeros(num_steps)
    measurements = np.zeros(num_steps)
    filtered_path = np.zeros(num_steps)

    # Perform simulation
    for i in range(num_steps):
        # True motion
        u = np.random.normal(0, 1)  # Random acceleration
        x = np.dot(A, x) + np.dot(B, u)
        true_path[i] = x[0, 0]  # Update true path

        # Take a noisy measurement
        z = measurement_func(x)
        measurements[i] = z[0, 0]  # Update measurements

        # Perform Kalman filter prediction and update
        x, P = kalman_filter_prediction_update(x, P, z, Q, R, dt)
        filtered_path[i] = x[0, 0]  # Update filtered path

    # Calculate precision (covariance) and accuracy (mean squared error)
    precision = calculate_precision(P)
    accuracy = calculate_accuracy(true_path, filtered_path)

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(np.arange(0, num_steps * dt, dt), true_path, label='True Path', color='blue', linewidth=2)
    plt.plot(np.arange(0, num_steps * dt, dt), measurements, 'ro', label='Measurements', markersize=3)
    plt.plot(np.arange(0, num_steps * dt, dt), filtered_path, label='Filtered Path', color='green', linewidth=2)
    plt.title('1D Hallway Robot Localization with Kalman Filter')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Position')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Print precision and accuracy
    print("Precision (Covariance):", precision)
    print("Accuracy (Mean Squared Error):", accuracy)

# Interactive widget for noise level adjustment
interact(simulate_robot, Q_value=FloatSlider(min=0.1, max=2, step=0.1, value=1, description='Q (Process Noise)'),
                         R_value=FloatSlider(min=1, max=20, step=1, value=5, description='R (Measurement Noise)'))



interactive(children=(FloatSlider(value=1.0, description='Q (Process Noise)', max=2.0, min=0.1), FloatSlider(v…

<function __main__.simulate_robot(Q_value, R_value)>